In [ ]:
import time
from playsound import playsound

import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [ ]:
# Function -----------------------------------------------------------------------------------------------------
def name_to_str(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

def find_index(index, pred_type):
    equal = np.empty(0)
    for i in np.arange(0, pred_type.size):
        count = index.count(pred_type[i])
        if count == 0:
            equal = np.append(equal, "False")
        else:
            equal = np.append(equal, "True")
    return pd.DataFrame({"equal":equal})

def result(test_x, test_y):
    # Prediction
    pred = model.predict(test_x)
    truth_type = test_y.argmax(axis = 1)
    pred_type = pred.argmax(axis = 1)
    
    # Accuracy of single
    single_accuracy = round(model.evaluate(test_x, test_y)[1], 3)
    print("\n★--- Single_accuracy:", single_accuracy)
    print("\n------------------------------------------------------------------")
    
    # Get Index
    name = name_to_str(pred_x,globals())[0]
    if name == "x_test_normal": class_index = normal_index; total_index = normal_index
    if name == "x_test_speeding": class_index = speeding_index; total_index = aberrant_index
    if name == "x_test_cheating": class_index = cheating_index; total_index = aberrant_index
    if name == "x_test_r_guessing": class_index = r_guessing_index; total_index = aberrant_index
    # print(class_index, total_index)
    
    # Accuracy of class
    class_df = find_index(class_index, pred_type)
    if class_df.value_counts().shape[0] != 1:
        class_accuracy = round(class_df.value_counts()["True"] / class_df.size, 3)
    else:
        if class_df.loc[0]["equal"] == "True":
            class_accuracy = 1
        else:
            class_accuracy = 0

    print("\n★--- Class_accuracy:", class_accuracy)
    print("\n------------------------------------------------------------------")
    
    # Accuracy of total
    total_df = find_index(total_index, pred_type)
    if total_df.value_counts().shape[0] != 1:
        total_accuracy = round(total_df.value_counts()["True"] / total_df.size, 3)
    else:
        if total_df.loc[0]["equal"] == "True":
            total_accuracy = 1
        else:
            total_accuracy = 0
    print("\n★--- Total_accuracy:", total_accuracy)
    print("\n------------------------------------------------------------------")
    
    return pred_type, single_accuracy, class_accuracy, total_accuracy

In [ ]:
print(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime()))
data_type = ["T", "O", "F"]

for hasRT in data_type:
    
    ########## Data parameters ##########
    start_name = "Q_6_40_high_0.2"
    #####################################
    
    # Generate arrays to save the results -----------------
    normal_single_accuracy = np.empty(0)
    speeding_single_accuracy = np.empty(0)
    cheating_single_accuracy = np.empty(0)
    r_guessing_single_accuracy = np.empty(0)

    normal_class_accuracy = np.empty(0)
    speeding_class_accuracy = np.empty(0)
    cheating_class_accuracy = np.empty(0)
    r_guessing_class_accuracy = np.empty(0)

    normal_total_accuracy = np.empty(0)
    speeding_total_accuracy = np.empty(0)
    cheating_total_accuracy = np.empty(0)
    r_guessing_total_accuracy = np.empty(0)

    pred_data = np.empty(0)
    
    for rs in np.arange(1, 31):
        # Repeat 30 times (1:31) -----------------------------
        seed = str(rs)
        print("--------------------- running " + seed + " ---------------------")
        
        # Read the data ---------------------
        train_x_path = "./DATA2/" + seed + "s_" + start_name + "_train_x_" + hasRT + ".csv"
        train_y_path = "./DATA2/"  + seed + "s_" + start_name + "_train_y.csv"

        test_normal_x_path = "./DATA2/"  + seed + "s_" + start_name + "_test_normal_x_" + hasRT + ".csv"
        test_normal_y_path = "./DATA2/"  + seed + "s_" + start_name + "_test_normal_y.csv"
        test_cheating_x_path = "./DATA2/"  + seed + "s_" + start_name + "_test_cheating_x_" + hasRT + ".csv"
        test_cheating_y_path = "./DATA2/"  + seed + "s_" + start_name + "_test_cheating_y.csv"
        test_speeding_x_path = "./DATA2/"  + seed + "s_" + start_name + "_test_speeding_x_" + hasRT + ".csv"
        test_speeding_y_path = "./DATA2/"  + seed + "s_" + start_name + "_test_speeding_y.csv"
        test_r_guessing_x_path = "./DATA2/"  + seed + "s_" + start_name + "_test_r_guessing_x_" + hasRT + ".csv"
        test_r_guessing_y_path = "./DATA2/"  + seed + "s_" + start_name + "_test_r_guessing_y.csv"

        x_train = np.array(pd.read_csv(train_x_path).drop("Unnamed: 0", axis=1))
        y_train = np.array(pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1))

        x_test_normal = np.array(pd.read_csv(test_normal_x_path).drop("Unnamed: 0", axis=1))
        y_test_normal = np.array(pd.read_csv(test_normal_y_path).drop("Unnamed: 0", axis=1))
        x_test_cheating = np.array(pd.read_csv(test_cheating_x_path).drop("Unnamed: 0", axis=1))
        y_test_cheating = np.array(pd.read_csv(test_cheating_y_path).drop("Unnamed: 0", axis=1))
        x_test_speeding = np.array(pd.read_csv(test_speeding_x_path).drop("Unnamed: 0", axis=1))
        y_test_speeding = np.array(pd.read_csv(test_speeding_y_path).drop("Unnamed: 0", axis=1))
        x_test_r_guessing = np.array(pd.read_csv(test_r_guessing_x_path).drop("Unnamed: 0", axis=1))
        y_test_r_guessing = np.array(pd.read_csv(test_r_guessing_y_path).drop("Unnamed: 0", axis=1))

        # Get the index of the aberrant type ---------------
        def get_same_element_index(ob_list, word):
            return [i for (i, v) in enumerate(ob_list) if v == word]

        normal_list = pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1).columns.str.contains('normal').tolist()
        speeding_list = pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1).columns.str.contains('speeding').tolist()
        cheating_list = pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1).columns.str.contains('cheating').tolist()
        r_guessing_list = pd.read_csv(train_y_path).drop("Unnamed: 0", axis=1).columns.str.contains('r_guessing').tolist()

        normal_index = get_same_element_index(normal_list, True)
        aberrant_index = get_same_element_index(normal_list, False)
        speeding_index = get_same_element_index(speeding_list, True)
        cheating_index = get_same_element_index(cheating_list, True)
        r_guessing_index = get_same_element_index(r_guessing_list, True)


        # Training_Set and Verification_Set (5-5) ---
        x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,train_size=0.5,random_state=666)
        print("x_train.shape:", x_train.shape)
        print("y_train.shape:", y_train.shape)

        ########## Model parameter ##########
        layer_one = x_train.shape[1] - 1
        batch_size = int(x_train.shape[0] / 1000)
        learning_rate=0.0001
        epochs = 100
        #####################################

        # Build model -----------------------
        model = Sequential()
        model.add(Dense(units = layer_one, input_dim = x_train.shape[1], activation = "relu"))
        model.add(Dense(units = y_train.shape[1], activation = "softmax"))

        adam = Adam(learning_rate = learning_rate)

        model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics = ['accuracy'])

        # Training model ------------------------
        H = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), workers=20, use_multiprocessing=True)

        # Results of normal ----------------------
        pred_x = x_test_normal
        pred_y = y_test_normal
        print(name_to_str(pred_x,globals()))
        normal_pred_type, single_accuracy, class_accuracy, total_accuracy = result(pred_x, pred_y)
        normal_single_accuracy = np.append(normal_single_accuracy, single_accuracy)
        normal_class_accuracy = np.append(normal_class_accuracy, class_accuracy)
        normal_total_accuracy = np.append(normal_total_accuracy, total_accuracy)

        # Results of speeding --------------------
        pred_x = x_test_speeding
        pred_y = y_test_speeding
        print(name_to_str(pred_x,globals()))
        speeding_pred_type, single_accuracy, class_accuracy, total_accuracy = result(pred_x, pred_y)
        speeding_single_accuracy = np.append(speeding_single_accuracy, single_accuracy)
        speeding_class_accuracy = np.append(speeding_class_accuracy, class_accuracy)
        speeding_total_accuracy = np.append(speeding_total_accuracy, total_accuracy)

        # Results of cheating ---------------------
        pred_x = x_test_cheating
        pred_y = y_test_cheating
        print(name_to_str(pred_x,globals()))
        cheating_pred_type, single_accuracy, class_accuracy, total_accuracy = result(pred_x, pred_y)
        cheating_single_accuracy = np.append(cheating_single_accuracy, single_accuracy)
        cheating_class_accuracy = np.append(cheating_class_accuracy, class_accuracy)
        cheating_total_accuracy = np.append(cheating_total_accuracy, total_accuracy)

        # Results of r_guessing -------------------
        pred_x = x_test_r_guessing
        pred_y = y_test_r_guessing
        print(name_to_str(pred_x,globals()))
        r_guessing_pred_type, single_accuracy, class_accuracy, total_accuracy = result(pred_x, pred_y)
        r_guessing_single_accuracy = np.append(r_guessing_single_accuracy, single_accuracy)
        r_guessing_class_accuracy = np.append(r_guessing_class_accuracy, class_accuracy)
        r_guessing_total_accuracy = np.append(r_guessing_total_accuracy, total_accuracy)
        
        pred_data = np.concatenate([pred_data, normal_pred_type, speeding_pred_type, cheating_pred_type, r_guessing_pred_type])
        
        
    print("\n-------------------  ending...  ", hasRT)
    
    # Result sorting
    res_dataFrame = pd.DataFrame({"normal_single_accuracy":normal_single_accuracy,
                              "speeding_single_accuracy":speeding_single_accuracy,
                              "cheating_single_accuracy":cheating_single_accuracy,
                              "r_guessing_single_accuracy":r_guessing_single_accuracy,
                              "normal_class_accuracy":normal_class_accuracy,
                              "speeding_class_accuracy":speeding_class_accuracy,
                              "cheating_class_accuracy":cheating_class_accuracy,
                              "r_guessing_class_accuracy":r_guessing_class_accuracy,
                              "normal_total_accuracy":normal_total_accuracy,
                              "speeding_total_accuracy":speeding_total_accuracy,
                              "cheating_total_accuracy":cheating_total_accuracy,
                              "r_guessing_total_accuracy":r_guessing_total_accuracy})

    pred_dataFrame = pd.DataFrame(pred_data, columns=["normal_speeding_cheating_guessing"])

    # Export Results
    res_dataFrame.to_csv(path_or_buf = "./res2/" + start_name + "_" + hasRT + ".csv", index=False)
    pred_dataFrame.to_csv(path_or_buf = "./pred2/" + start_name + "_" + hasRT + ".csv", index=False)

print(time.strftime("%Y/%m/%d %H:%M:%S", time.localtime()))
playsound("Clew tone.mp3")
playsound("Clew tone.mp3")